The model trained in this notebook is the best performing model I was able to get (on the task of MovieLens rating prediction) after a bit of experimentation with hyperparameters. It uses:
- matrix factorization architecture
- embedding size = 32
- embedding L2 penalty
- dropout (applied to embedding vectors)

It's used for lessons 3 and 4 of the embeddings course, which are on exploring learned embeddings with Gensim and t-SNE, respectively. (On the assumption that the model with the best error will probably have the 'best' embeddings, in terms of identifying interesting/useful latent properties.

TODO: I imagine there could still be some further gains with a little more experimentation.
- Training and val errors are so close, I wonder about cutting dropout prob in half or something
- And/or doubling embedding size
- Adding biases back in?

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import random

RUNNING_ON_KERNELS = 'KAGGLE_WORKING_DIR' in os.environ
input_dir = '../input' if RUNNING_ON_KERNELS else '../input/movielens_preprocessed'
ratings_path = os.path.join(input_dir, 'rating.csv')
df = pd.read_csv(ratings_path, usecols=['userId', 'movieId', 'rating', 'y'])

tf.set_random_seed(1); np.random.seed(1); random.seed(1)

In [ ]:
movie_embedding_size = user_embedding_size = 32
user_id_input = keras.Input(shape=(1,), name='user_id')
movie_id_input = keras.Input(shape=(1,), name='movie_id')
movie_r12n = keras.regularizers.l1_l2(l1=0, l2=1e-6)
user_r12n = keras.regularizers.l1_l2(l1=0, l2=1e-7)
dropout = .2
user_embedded = keras.layers.Embedding(df.userId.max()+1, user_embedding_size,
                                       embeddings_initializer='glorot_uniform',
                                       embeddings_regularizer=user_r12n,
                                       input_length=1, name='user_embedding')(user_id_input)
user_embedded = keras.layers.Dropout(dropout)(user_embedded)
movie_embedded = keras.layers.Embedding(df.movieId.max()+1, movie_embedding_size, 
                                        embeddings_initializer='glorot_uniform',
                                        embeddings_regularizer=movie_r12n,
                                        input_length=1, name='movie_embedding')(movie_id_input)
movie_embedded = keras.layers.Dropout(dropout)(movie_embedded)

dotted = keras.layers.Dot(2)([user_embedded, movie_embedded])
out = keras.layers.Flatten()(dotted)

biases = 0
if biases:
    bias_r12n = None
    bias_r12n = keras.regularizers.l1_l2(l1=1e-4, l2=1e-7) # XXX 1e-6 -> 1e-4
    bias_init = 'zeros'
    movie_b = keras.layers.Embedding(df.movieId.max()+1, 1, 
                                             name='movie_bias',
                                             embeddings_initializer=bias_init,
                                             embeddings_regularizer=bias_r12n,
                                            )(movie_id_input)
    movie_b = keras.layers.Flatten()(movie_b)

    user_b = keras.layers.Embedding(df.userId.max()+1, 1, 
                                             name='user_bias',
                                             embeddings_initializer=bias_init,
                                             embeddings_regularizer=bias_r12n,
                                            )(user_id_input)
    user_b = keras.layers.Flatten()(user_b)
    out = keras.layers.Add()([user_b, movie_b, out])

model = keras.Model(
    inputs = [user_id_input, movie_id_input],
    outputs = out,
)
model.compile(
    tf.train.AdamOptimizer(0.001),
    loss='MSE',
    metrics=['MAE'],
)

tf.set_random_seed(1); np.random.seed(1); random.seed(1)
history = model.fit(
    [df.userId, df.movieId],
    df.y,
    batch_size=10**4,
    epochs=30,
    verbose=2,
    validation_split=.05,
);

Train on 19000249 samples, validate on 1000014 samples
Epoch 1/10
 - 217s - loss: 0.8603 - mean_absolute_error: 0.7153 - val_loss: 0.7418 - val_mean_absolute_error: 0.6529
Epoch 2/10
 - 209s - loss: 0.7306 - mean_absolute_error: 0.6472 - val_loss: 0.7112 - val_mean_absolute_error: 0.6358
Epoch 3/10
 - 213s - loss: 0.7061 - mean_absolute_error: 0.6342 - val_loss: 0.6933 - val_mean_absolute_error: 0.6261
Epoch 4/10
 - 202s - loss: 0.6885 - mean_absolute_error: 0.6249 - val_loss: 0.6792 - val_mean_absolute_error: 0.6184
Epoch 5/10
 - 205s - loss: 0.6744 - mean_absolute_error: 0.6175 - val_loss: 0.6688 - val_mean_absolute_error: 0.6126
Epoch 6/10
 - 184s - loss: 0.6635 - mean_absolute_error: 0.6116 - val_loss: 0.6614 - val_mean_absolute_error: 0.6083
Epoch 7/10
 - 197s - loss: 0.6547 - mean_absolute_error: 0.6068 - val_loss: 0.6559 - val_mean_absolute_error: 0.6049
Epoch 8/10
 - 197s - loss: 0.6480 - mean_absolute_error: 0.6031 - val_loss: 0.6518 - val_mean_absolute_error: 0.6024
Epoch 9/1

In [ ]:
model.save('movie_svd_model_32.h5')